In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import os

In [2]:
spark = SparkSession \
    .builder \
    .getOrCreate()

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/30 21:29:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
os.listdir('/standard/ds7200-apt4c/isaac_yuyang_final_project/')

['Combined_Flights_2020.parquet',
 'Combined_Flights_2019.parquet',
 'Combined_Flights_2022.parquet',
 'flights_2018.parquet',
 'Combined_Flights_2018.parquet',
 'Combined_Flights_2021.parquet',
 'Airlines.csv']

In [9]:
airlines = spark.read.load('/standard/ds7200-apt4c/isaac_yuyang_final_project/Airlines.csv', header=True, format='csv', sep=',')

In [10]:
airlines.show(10)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 02Q|       Titan Airways|
| 04Q|  Tradewind Aviation|
| 05Q| Comlux Aviation, AG|
| 06Q|Master Top Linhas...|
| 07Q| Flair Airlines Ltd.|
| 09Q|      Swift Air, LLC|
| 0BQ|                 DCA|
| 0CQ|ACM AIR CHARTER GmbH|
| 0GQ|Inter Island Airw...|
| 0HQ|Polar Airlines de...|
+----+--------------------+
only showing top 10 rows



In [12]:
airlines.count()

1571

In [7]:
flights_2018 = spark.read.parquet('/standard/ds7200-apt4c/isaac_yuyang_final_project/flights_2018.parquet')

In [10]:
flights_2018.columns

['FlightDate',
 'Airline',
 'Origin',
 'Dest',
 'Cancelled',
 'Diverted',
 'CRSDepTime',
 'DepTime',
 'DepDelayMinutes',
 'DepDelay',
 'ArrTime',
 'ArrDelayMinutes',
 'AirTime',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'Distance',
 'Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Operating_Airline',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRS

In [30]:
flights_2018.groupBy('OriginCityName').count().show()

+------------------+-----+
|    OriginCityName|count|
+------------------+-----+
|   Gainesville, FL| 2092|
|      Richmond, VA|18967|
|       Ontario, CA|18590|
|        Tucson, AZ|13103|
|     Pago Pago, TT|  122|
|  Myrtle Beach, SC| 8961|
|       Medford, OR| 6696|
|  Palm Springs, CA| 8041|
|       Redding, CA| 1169|
|     St. Cloud, MN|  145|
|       Durango, CO| 2851|
|   Devils Lake, ND|  626|
|      Gustavus, AK|   31|
|        Mobile, AL| 4027|
|Corpus Christi, TX| 4291|
|       Dubuque, IA|  435|
|      Columbus, GA|  606|
|    Huntsville, AL| 6156|
|    Fort Myers, FL|25611|
|     Pensacola, FL| 7248|
+------------------+-----+
only showing top 20 rows



In [27]:
numeric_cols = [field.name for field in flights_2018.schema.fields 
                if field.dataType.typeName() in ['integer', 'double', 'float', 'long']]
correlations = []
for col in numeric_cols:
    if col != 'ArrDelay':
        corr = flights_2018.stat.corr(col, 'ArrDelay')
        correlations.append((col, corr))
# Sort by absolute correlation value
correlations.sort(key=lambda x: abs(x[1]), reverse=True)

# Display results
for col, corr in correlations:
    print(f"{col}: {corr:.4f}")

ArrDelayMinutes: 0.9815
DepDelay: 0.9541
DepDelayMinutes: 0.9510
Year: nan
ArrivalDelayGroups: 0.8792
DepartureDelayGroups: 0.8300
ArrDel15: 0.6155
DepDel15: 0.5735
TaxiOut: 0.2128
DepTime: 0.1286
WheelsOff: 0.1233
TaxiIn: 0.1033
CRSDepTime: 0.0945
CRSArrTime: 0.0826
WheelsOn: 0.0411
ActualElapsedTime: 0.0400
ArrTime: 0.0350
DOT_ID_Operating_Airline: 0.0339
DOT_ID_Marketing_Airline: 0.0327
DestWac: -0.0221
__index_level_0__: -0.0201
CRSElapsedTime: -0.0185
Flight_Number_Operating_Airline: 0.0177
Flight_Number_Marketing_Airline: 0.0177
DistanceGroup: -0.0166
Distance: -0.0166
OriginWac: -0.0128
DayOfWeek: -0.0103
DayofMonth: -0.0095
OriginCityMarketID: -0.0082
DestStateFips: 0.0077
OriginAirportSeqID: -0.0040
OriginAirportID: -0.0040
DivAirportLandings: -0.0036
AirTime: 0.0031
DestCityMarketID: -0.0022
OriginStateFips: 0.0020
DestAirportID: 0.0005
DestAirportSeqID: 0.0005
Quarter: 0.0004
Month: 0.0002
